In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import numpy as np            
import urllib.parse
import random
import math
from shapely.geometry import Point
from shapely import wkb


from conf import *

def get_db_engine():
    db_pass_enc = urllib.parse.quote_plus(DB_PASS)
    return create_engine(f'postgresql://{DB_USER}:{db_pass_enc}@{DB_HOST}:{DB_PORT}/{DB_NAME}')



def get_query_result(query, db_conn, params=[]):
    return pd.read_sql(query, db_conn, params=params)
        

def get_garmin_user_id(db_conn, pattern=None):
    # query = f"SELECT user_id FROM {DB_USER_TABLE} WHERE device LIKE '%Garmin%'"
    query = f"SELECT user_id FROM {DB_USER_TABLE} WHERE device='Garmin SQ'"
    params = []
    if pattern:
        query += f" AND email LIKE %s"
        pattern = f'%{pattern}%'
        params = [pattern]
    # execute the query
    return get_query_result(query, db_conn, params)


def get_series(db_conn, user_ids, start_time, table_name):
    query = f'''
    SELECT * FROM {table_name}
    WHERE user_id in ({','.join(['%s']*len(user_ids))}) AND timestamp >= '{start_time}' 
    ORDER BY timestamp ASC;
    '''
    params = user_ids
    return get_query_result(query, db_conn, params=params)

In [2]:
db_conn = get_db_engine()
garmin_user_ids = get_garmin_user_id(db_conn, 'pps')
hrates = get_series(db_conn, garmin_user_ids.values.squeeze(), START_TIME, DB_HEART_TABLE)

In [3]:
sample_user_id = garmin_user_ids.values[0, 0]
timestamps = hrates.loc[hrates['user_id']==sample_user_id]['timestamp'].values
timestamps

array(['2022-10-12T12:53:15.000000000', '2022-10-12T12:53:30.000000000',
       '2022-10-12T12:53:45.000000000', '2022-10-12T12:54:00.000000000',
       '2022-10-12T12:54:15.000000000', '2022-10-12T12:54:30.000000000',
       '2022-10-12T12:54:45.000000000', '2022-10-12T12:55:00.000000000',
       '2022-10-12T12:55:15.000000000', '2022-10-12T12:55:30.000000000',
       '2022-10-12T12:55:45.000000000', '2022-10-12T12:56:00.000000000',
       '2022-10-12T12:56:15.000000000', '2022-10-12T12:56:30.000000000',
       '2022-10-12T12:56:45.000000000', '2022-10-12T12:57:00.000000000',
       '2022-10-12T12:57:15.000000000', '2022-10-12T12:57:30.000000000',
       '2022-10-12T12:57:45.000000000', '2022-10-12T12:58:00.000000000',
       '2022-10-12T12:58:15.000000000', '2022-10-12T12:58:30.000000000',
       '2022-10-12T12:58:45.000000000', '2022-10-12T12:59:00.000000000',
       '2022-10-12T12:59:15.000000000', '2022-10-12T12:59:30.000000000',
       '2022-10-12T12:59:45.000000000', '2022-10-12

In [4]:
import random
import math
import folium

# Define the USC location as a latitude and longitude
usc_center_y = 34.0224
usc_center_x = -118.2851

# Define a radius in meters around the USC location
usc_radius = 10000

# Define the Earth's radius in meters
R = 6371000

# Define the number of people
num_people = len(garmin_user_ids)

# Define the time interval for each point in seconds
time_interval = 15

# Maximum distance at each interval
max_dist = 10000

# Define the number of timesteps
timesteps = timestamps.shape[0]
 

# Generate trajectories for each person
people = []
for i in range(num_people):
    # Choose a random starting point within the radius of the USC location
    r = usc_radius / math.sqrt(random.uniform(0, 1))
    theta = random.uniform(0, 2*math.pi)
    start_x = usc_center_x + (r / R) * math.cos(theta)
    start_y = usc_center_y + (r / R) * math.sin(theta)
    start = (start_y, start_x)
    
    # Generate a random distance and direction for each time interval
    # to simulate a person's movement over time
    trajectory = [start]
    for j in range(timesteps - 1):
        # Choose a random distance between 0 and max_dist meters
        distance = random.uniform(0, max_dist)
        
        # Choose a random direction
        direction = random.uniform(0, 2*math.pi)
        
        # Calculate the latitude and longitude endpoint of the movement
        lat1, lon1 = trajectory[-1]
        lat2 = lat1 + (distance / R) * math.cos(direction)
        lon2 = lon1 + (distance / R) * math.sin(direction)
        endpoint = (lat2, lon2)
        
        # Add the endpoint to the person's trajectory
        trajectory.append(endpoint)
        
    # Store the person's information in a dictionary
    person = {
        'start': start,
        'trajectory': trajectory
    }
    
    # Add the person's information to the list of people
    people.append(person)

# Define the center of the map as the USC location
usc_location = (usc_center_y, usc_center_x)

# Create a map centered at the USC location
m = folium.Map(location=usc_location, zoom_start=15)

# Choose which person's full trajectory to display (between 0 and num_people - 1)
person_index = 0

# Add markers for the starting and ending points of the person's trajectory
start = people[person_index]['start']
end = people[person_index]['trajectory'][-1]
folium.Marker(location=start, icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=end, icon=folium.Icon(color='red')).add_to(m)

# Add a line representing the person's trajectory
trajectory = people[person_index]['trajectory']
folium.PolyLine(locations=trajectory, color='blue').add_to(m)

# Display the map
m

In [5]:
df = pd.DataFrame()
for i, person in enumerate(people):
    traj = person['trajectory']
    person_id = garmin_user_ids.user_id.values[i]
    dframe = pd.DataFrame({'user_id': [person_id]*len(traj), 
                           'timestamp': timestamps,
                           'coordinates': [Point(x[1], x[0]) for x in traj]                           }
                          )
    # save it to db as point
    df = pd.concat([df, dframe])
df.set_index(['timestamp'], inplace=True)
df.sort_index(inplace=True)
df

,user_id,coordinates
timestamp,,
2022-10-12 12:53:15,pps1@usc.edu,POINT (-118.2867900211667 34.02297683425875)
2022-10-12 12:53:15,pps8@usc.edu,POINT (-118.2853734219286 34.0205808648757)
2022-10-12 12:53:15,pps7@usc.edu,POINT (-118.2820988447544 34.02080809962413)
2022-10-12 12:53:15,pps6@usc.edu,POINT (-118.2869965661398 34.01942350667155)
2022-10-12 12:53:15,pps5@usc.edu,POINT (-118.2833331197561 34.02168801248575)
...,...,...
2022-10-12 15:11:00,pps3@usc.edu,POINT (-118.2776352820632 34.03716700741748)
2022-10-12 15:11:00,pps2@usc.edu,POINT (-118.2870033794053 34.01518901186508)
2022-10-12 15:11:00,pps1@usc.edu,POINT (-118.2621366891066 34.02599076655914)


In [21]:
# create table
with db_conn.connect() as conn:
    conn.execute(f'DROP TABLE IF EXISTS {DB_LOCATIONS_TABLE}')
    conn.execute(f'CREATE TABLE IF NOT EXISTS {DB_LOCATIONS_TABLE} (user_id text, timestamp timestamp, coordinates geometry(Point, 4326))')
    for timestamp, row in df.iterrows():
        db_conn.execute(f"INSERT INTO {DB_LOCATIONS_TABLE} (user_id, timestamp, coordinates) VALUES ('{row['user_id']}', '{timestamp}', ST_GeomFromText('{row['coordinates'].wkt}', 4326));")
        # print(f"INSERT INTO {DB_LOCATIONS_TABLE} (user_id, timestamp, coordinates) VALUES ('{row['user_id']}', '{timestamp}', ST_GeomFromText('{row['coordinates'].wkt}', 4326));")